In [62]:
!pip install ydata-profiling
!pip install plotnine
!pip install geopy
!pip install folium

^C
^C


In [65]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import numpy as np
from datetime import datetime

def merge_datasets_and_save(first_csv_path, second_csv_path, output_csv_path):
    """
    1)BASIC INFO:
    VE FUNKCI MĚŇTE POUZE NÁZEV SLOUPCE TABULKY ADDITIONAL INFO, KTERÝ VSTOUPÍ NA JOIN (additional_info.rename(columns={'listing_id': 'id'}),
    NÁZEV SLOUPCE NA JOIN  (merged_df = pd.merge(model_pop, additional_info, on='id', how='left'))
    A POČET ZÁZNAMŮ MODELOVÉ POPULACE PODLE POTŘEBY (model_pop = model_pop.iloc[:1000])
    V ČÁSTI def create_features(merged_df) MŮŽETE KODIT CO POTŘEBUJETE PODLE INSTRUKCÍ 
    PO VŠECH ÚPRAVÁCH ZAVOLEJTE FUNKCI S JEJÍMI PARAMETRY - SEKCE 3
    
    2)POPIS FUNKCE:
    Načte dva CSV soubory model_pop(calendar) a additional_info(listings, reviews), přejmenuje sloupec v additional_info dle potřeby, provede left join model_pop 
    (omezeného na prvních X záznamů pokud chcete) s additional_info na základě zadaného sloupce(zde listing_id)
    a uloží výsledek do výsledného features CSV souboru.

    3)PARAMETRY:
    first_csv_path (str): Cesta k model_pop.
    second_csv_path (str): Cesta k additional_info 
    output_csv_path (str): Cesta, kam se má výsledný CSV soubor uložit (features dataframe)
    """
    # Načtení prvního datasetu a omezení na prvních X záznamů - X si volte jak chcete
    model_pop = pd.read_csv(first_csv_path)
    #model_pop = model_pop.iloc[:500]  

    # Načtení druhého datasetu a přejmenování sloupce
    additional_info = pd.read_csv(second_csv_path)
    additional_info.rename(columns={'id': 'listing_id'}, inplace=True)
    
    # Provedení left join prvního datasetu s druhým datasetem
    merged_df = pd.merge(model_pop, additional_info, on='listing_id', how='left')
    merged_df = merged_df.groupby('listing_id').sample(n=1)

    def create_features(merged_df):
        merged_df=merged_df[['listing_id',"name","host_since","host_location","host_response_time","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_verifications","host_has_profile_pic","host_identity_verified","neighbourhood_cleansed","adjusted_price","reviews_per_month","instant_bookable","last_review","has_availability","availability_30"]]
        merged_df['adjusted_price'] = merged_df['adjusted_price'].str.slice(start=1)
        merged_df['adjusted_price'] =merged_df['adjusted_price'].str.replace(',', '').astype(float)
        ### Ranking, Bathroom, Beds ftrs
        merged_df[["name",'ranking']] = merged_df['name'].str.split('★', expand=True)
        merged_df[["ranking",'bedroom',"beds","bath"]] = merged_df['ranking'].str.split('·', expand=True)
        mapping = {'within an hour': 0, 'within a few hours': 1, 'within a day': 2, 'a few days or more': 3}
        merged_df['host_response_time'] = merged_df['host_response_time'].replace(mapping)

        # Převede čísla na číselný typ
        merged_df['ranking'].fillna(0, inplace=True)
        merged_df['ranking'] = merged_df['ranking'].replace("New ", 0)
        merged_df['ranking'] = merged_df['ranking'].astype(float) 

        # Inicializace geokodéru
        geolocator = Nominatim(user_agent="my_geocoder",timeout=5)

        # Slovník pro ukládání cache geografických souřadnic
        coordinates_cache = {}

        # Funkce pro získání geografických souřadnic pro místo s využitím cache
        def get_coordinates_cached(location):
            # Pokud jsou souřadnice uloženy v cache, vrátí je
            if location in coordinates_cache:
                return coordinates_cache[location]
            # Jinak provede geokódování a uloží do cache
            else:
                geo_location = geolocator.geocode(location)
                if geo_location:
                    coordinates = (geo_location.latitude, geo_location.longitude)
                    coordinates_cache[location] = coordinates
                    return coordinates
                else:
                    return None

        # Výpočet vzdálenosti pro každý řádek datasetu s využitím cache
        def calculate_distance_cached(row):
            coords1 = get_coordinates_cached(row["neighbourhood_cleansed"])
            coords2 = get_coordinates_cached(row["host_location"])
            if coords1 and coords2:
                return geodesic(coords1, coords2).kilometers
            else:
                return None


        # Přidání sloupce s vypočtenými vzdálenostmi do prvních deseti řádků datasetu s využitím cache
        merged_df["Distance"] = merged_df.apply(calculate_distance_cached, axis=1)

        def create_verification_category(df):
            # Vytvoření nového sloupce "Verifikace"
            merged_df['Verification'] = ''

            # Podmínky pro přiřazení kategorie do sloupce "Verifikace"
            conditions = [
                (merged_df['host_has_profile_pic'] == "t") & (merged_df['host_identity_verified'] == "f"),
                (merged_df['host_has_profile_pic'] == "f") & (merged_df['host_identity_verified'] == "t"),
                (merged_df['host_has_profile_pic'] == "t") & (merged_df['host_identity_verified'] == "t")
            ]
            categories = ['only profile', 'only identity', 'Both']

            # Přiřazení kategorie na základě podmínek
            merged_df['Verification'] = pd.Categorical(
                np.select(conditions, categories, default='Unknown'),
                categories=categories,
                ordered=True
            )

        # Příklad použití funkce s příkladovým DataFrame

        create_verification_category(merged_df)

        # Předpokládáme, že sloupec 'host_since' je ve formátu 'YYYY-MM-DD'
        merged_df['host_since'] = pd.to_datetime(merged_df['host_since'])
        merged_df['last_review'] = pd.to_datetime(merged_df['last_review'])

        # Dnešní datum
        today = datetime.now()
        
        # Výpočet počtu měsíců od data 'host_since' k dnešnímu datu
        merged_df['months_since_hosting'] = (today.year - merged_df['host_since'].dt.year) * 12 + (today.month - merged_df['host_since'].dt.month)
        merged_df['last_review'] = (today.year - merged_df['last_review'].dt.year) * 12 + (today.month - merged_df['last_review'].dt.month)

        ## All the sanity checks
        mode_value = merged_df['host_response_time'].mode()[0]
        merged_df['host_response_time'].fillna(mode_value, inplace=True)
        merged_df['last_review'].fillna(0, inplace=True)
        merged_df['reviews_per_month'].fillna(0, inplace=True)
        mode_value = merged_df['host_is_superhost'].mode()[0]
        merged_df['host_is_superhost'].fillna(mode_value, inplace=True)
        
        #Poslední merge
        merged_df=merged_df[["listing_id","adjusted_price","host_response_time","host_is_superhost","ranking","Distance","Verification","reviews_per_month","instant_bookable","last_review","has_availability","availability_30"]]
        return merged_df

    features = create_features(merged_df)

    # Uložení výsledného datasetu do CSV souboru
    features.to_csv(output_csv_path, index=False)
    
    print(f"Data byla úspěšně spojena a uložena do souboru {output_csv_path}")

merge_datasets_and_save("Data\\raw\\calendar.csv", "Data\\raw\\listings.csv", "Data\\interim\\features -1-nodum.csv")

C:\Users\blank\AppData\Local\Temp\ipykernel_20572\1682765508.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['adjusted_price'] = merged_df['adjusted_price'].str.slice(start=1)
C:\Users\blank\AppData\Local\Temp\ipykernel_20572\1682765508.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['adjusted_price'] =merged_df['adjusted_price'].str.replace(',', '').astype(float)
C:\Users\blank\AppData\Local\Temp\ipykernel_20572\1682765508.py:43: SettingWithCopyWarning: 
A value is trying to b

C:\Users\blank\AppData\Local\Temp\ipykernel_20572\1682765508.py:124: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['last_review'].fillna(0, inplace=True)
C:\Users\blank\AppData\Local\Temp\ipykernel_20572\1682765508.py:125: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['reviews_per_month'].fillna(0, inplace=True)
C:\Users\bl

Data byla úspěšně spojena a uložena do souboru features -1-nodum.csv
